# Kalman filter
This is an implementation of the example Kalman filter: [ExKF.m](https://github.com/cybergalactic/MSS/blob/master/mssExamples/ExKF.m).

ExKF Discrete-time Kalman filter (KF) implementation demonstrating
how the "predictor-corrector representation" can be applied to a
linear model:

$dx/dt = A \cdot x + B \cdot u + E \cdot w$ <br>
$y = C \cdot x + v$ <br>
$x_k+1 = Ad \cdot x_k + Bd \cdot u_k + Ed \cdot w_k$ <br>
$y_k = Cd \cdot x_k + v_k$ <br>

$Ad = I + h \cdot A$, $Bd = h \cdot B$, $Cd = C$ and $Ed = h \cdot E$ (h = sampling time). 


The case study is a ship model in yaw with heading angle measured at frequency
*f_m* [Hz], which can be chosen smaller or equal to the sampling frequency 
*f_s* [Hz]. The ratio between the frequencies must be a non-negative integer:
Integer:  Z = f_s/f_m >= 1  

Author:    Thor I. Fossen <br>
Date:      17 Oct 2018 <br>
Revisions: 28 Feb 2020, minor updates of notation <br>
           29 Mar 2020, added ssa(), new logic for no measurements <br>


In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv

import vessel_manoeuvring_models.visualization.book_format as book_format
book_format.set_style()

## Discrete-time Kalman Filter
Linear Time Variant state space model:<br>
$ \dot{x} = A(t)x + B(t)u + E(t)w $ <br>
where $w$ is zero mean Gausian process noise.

With measurements:<br>
$ y = C(t)x + D(t)u + \epsilon $ <br>
where $\epsilon$ is zero mean Gausian measurement noise.

### Model parameters for mass-damper system (x1 = yaw angle, x2 = yaw rate)

In [ ]:
A = np.array(
    [[0,    1],
     [0, -0.1]])

B = np.array(
    [[0],
     [1]]
)

E = np.array(
    [[0],
    [1]],
)
    
C = np.array([[1, 0]])

## Simulate system without Kalman Filtering
There is an control input to the system which is modelled as a sinus, this could be external disturbances from wind or waves: <br>
$ u = 0.1 \sin(0.1 t) $

In [ ]:
# simulation parameters
N  = 100   # no. of iterations
f_m = 1    # sampling frequency [Hz]
h_m  = 1/f_m  # sampling time: h  = 1/f_s (s) 
t = np.arange(0,N*h_m,h_m)

In [ ]:
# initial values for x and u
x = np.array([[0, 0]]).T	        
us = 0.1 * np.sin(0.1*t) # inputs
np.random.seed(42)
ws = 0.1 * np.random.normal(scale=1, size=N)  # process noise


In [ ]:
x_prd = x
simdata = []
for i in range(N):
    
    u = us[i] # input
    w = ws[i] # process noise
    
    x_dot = A @ x + B * u + E * w;
       
    ## Euler integration (k+1)
    x = x + h_m * x_dot
    
    simdata.append(x.flatten())

simdata = np.array(simdata)
df = pd.DataFrame(simdata, columns=['yaw','yaw rate'], index=t)
df['u'] = us
df['w'] = ws

In [ ]:
fig,axes=plt.subplots(nrows=3)
df.plot(y='u', label='u (input)', ax=axes[0])
df.plot(y='yaw', ax=axes[1])
df.plot(y='yaw rate', ax=axes[2]);

## Discretisized system matrixes

$A_d[k] = \Phi$ <br>
$B_d[k] = A^{-1}(\Phi-I)B$ <br>
$C_d[k] = C$ <br>
$D_d[k] = D$ <br>
$E_d[k] = A^{-1}(\Phi-I)E$ <br>

The trasition matrix is calculated as a taylor expansion: <br>
$\Phi \approx I + A h + \frac{1}{2}(A h)^2 + ... + \frac{1}{N!}(A h)^N$ <br>
choosing $N=1$:<br>
$A_d[k]=I + A h$

In [ ]:
df['epsilon'] = epsilons = 0.1 * np.random.normal(scale=1, size=N)  # measurement noise
df['y'] = df['yaw'] + df['epsilon']
ys = np.zeros((N,2))
ys[:,0] = df['y'].values

In [ ]:
ys.shape

In [ ]:
f_s = 10  # sampling frequency [Hz]
h = 1 / f_s  # sampling time: h  = 1/f_s (s)
Ad = np.eye(2) + h * A
Bd = h * B
Cd = C
Ed = h * E

In [ ]:
# initial values for x and u
x = np.array([[0, 0]]).T      

In [ ]:
# initialization of Kalman filter
x_prd = np.array([[0, 0]]).T        
P_prd = np.diag([1, 1])
Qd = 1
Rd = 10

In [ ]:
def ssa(angle):
    """
    maps an angle in rad to the interval [-pi pi]
    """
    return np.mod( angle + np.pi, 2 * np.pi ) - np.pi 

In [ ]:
def filter(
    x0,
    P_prd,
    h_m,
    h,
    us,
    ws,
    ys,
    Ad,
    Bd,
    Cd,
    Ed,
    Qd,
    Rd,
):
    x_prd = x0
    t = 0
    df = pd.DataFrame()

    for i in range(len(us)):

        u = us[i]  # input
        w = ws[i]  # process noise
        y = ys[i].T  # measurement

        for j in range(int(h_m / h)):
            t += h
            # Compute kalman gain:
            S = Cd @ P_prd @ Cd.T + Rd  # System uncertainty
            K = P_prd @ Cd.T @ inv(S)
            IKC = np.eye(2) - K @ Cd

            # State corrector:
            x_hat = x_prd + K * np.rad2deg(
                ssa(np.deg2rad(y - Cd @ x_prd))
            )  # smallest signed angle
            
            # corrector
            P_hat = IKC * P_prd @ IKC.T + K * Rd @ K.T

            # Predictor (k+1)
            x_prd = Ad @ x_hat + Bd * u
            P_prd = Ad @ P_hat @ Ad.T + Ed * Qd @ Ed.T

            s = pd.Series(name=t)
            s["yaw"] = x_hat[0][0]
            s["yaw rate"] = x_hat[1][0]
            s["yaw predictor"] = x_prd[0][0]
            s["yaw rate predictor"] = x_prd[1][0]

            df = df.append(s)

    return df

In [ ]:
# initialization of Kalman filter
x0 = np.array([[10, 0]]).T
P_prd = np.diag([1, 1])
Qd = 1
Rd = 10
df2 = filter(x0=x0, P_prd=P_prd, h_m=h_m, h=h, us=us, ws=ws, ys=ys, Ad=Ad, Bd=Bd, 
             Cd=Cd, Ed=Ed, Qd=Qd, Rd=Rd)

In [ ]:
df2.head()

In [ ]:
fig,axes=plt.subplots(nrows=3)
df.plot(y='u', label='u (input)', ax=axes[0])
axes[1].set_ylabel('yaw')
df.plot(y='y', style='.', alpha=0.7, ax=axes[1])
df.plot(y='yaw', label='model', ax=axes[1])
df2.plot(y='yaw predictor', label='predictor', style='--', ax=axes[1])
df2.plot(y='yaw', label='kalman', style=':', ax=axes[1])


axes[2].set_ylabel('yaw rate')
df.plot(y='yaw rate', label='model', ax=axes[2]);
df2.plot(y='yaw rate predictor', label='predictor', style='--', ax=axes[2]);
df2.plot(y='yaw rate', label='kalman', style=':', ax=axes[2])
